# Mount Baker CSM stereo run (with core 128 km^2 dataset)
- Shashank Bhushan and Seth Vanderwilt
- May 2023

- Demonstration of the workflow to create a stack of inputs for elevation refinement models- this notebook shows execution with Mount Baker 20150911 stereo pair based on experiments with May 2019 South Cascade Glacier stereo processing.
- Remaining issue
  * Alignment with bare ground lidar brings stereo rasters close to lidar DEM but they do not align perfectly. Investigate and fix - previous Baker processing script (`pc_laz_prep_full.sh`) resulted in lidar DEM and stereo DEM lining up perfectly.
- Next steps:
  * Remove remaining hardcoding of filenames / ASP outputs / CRS
  * Set reasonable bounds based on stereo pair & lidar intersection
  * Pull these revised commands back into generic stereo script rather than having to execute in notebook
  * (In script) final steps of copying stack of desired rasters into single folder

In [ ]:
pair_dir = "/mnt/1.0_TB_VOLUME/sethv/shashank_data/WV01_20150911_1020010042D39D00_1020010043455300"


## Find the left and right images based on XML metadata

In [ ]:
import os
import xml.etree.ElementTree as ET
import glob
import rioxarray

In [ ]:
xml_pattern = os.path.join(pair_dir, "*.xml")
image_gsds = []
for xml_fn in glob.glob(xml_pattern):
    root = ET.parse(xml_fn)
    # Extract mean GSD from the metadata fields
    meanGSD = round(float(root.find("./IMD/IMAGE/MEANPRODUCTGSD").text),3)

    print(os.path.basename(xml_fn), meanGSD)
    image_id = os.path.basename(xml_fn).split(".")[0]
    
    # Extract fields of interest and just keep all metadata around as 'xml_root'
    image_dict = {
        "xml_fn": xml_fn,
        "image_fn": xml_fn.replace("xml", "tif"),
        "meanGSD": meanGSD,
        "image_id": image_id,
        "xml_root": root
    }
    
    image_gsds.append(image_dict)
    print(image_id)

left_dict = min(image_gsds, key=lambda i: i["meanGSD"])
right_dict = max(image_gsds, key=lambda i: i["meanGSD"])
res = min(image_gsds, key=lambda i: i["meanGSD"])["meanGSD"]
print(f"Highest resolution to use for orthoimages & stereo: {res} meters")

left_image_id = left_dict["image_id"]
right_image_id = right_dict["image_id"]

## TODO: Determine common intersection extent
* Better way: extract URLON ULLAT etc. fields from the XML and find the shared bbox between the images and lidar point cloud to use for rest of dataset creation.

In [ ]:
xmin, ymin, xmax, ymax = (583000, 5395000, 591000, 5411000)  # bounds in reprojected CRS


## Bundle adjustment

In [ ]:
!bundle_adjust --ip-per-image 10000 -t dg --dg-use-csm \
    --camera-weight 0 --tri-weight 0.1 --tri-robust-threshold 0.1 \
    $left_image_fn $right_image_fn $left_image_xml $right_image_xml \
    -o dg_csm_model_refined/run

## Register camera models to reference DEM
* Using Copernicus GLO-30 Digital Elevation Model from [OpenTopography](https://portal.opentopography.org/raster?opentopoID=OTSDEM.032021.4326.3)
* Coordinate System:
  - Horizontal: WGS84 [EPSG: 4326]
  - Vertical: EGM2008 [EPSG: 3855] 

In [ ]:
refdem_from_opentopography = "output_COP30.tif"
refdem = "refdem_copernicus_30m_32610_wgs84.tif" # reprojected & cropped to extent

## Adjust reference DEM to desired datum.
* Call `dem_geoid` - but only in newer ASP versions with bugfix for .jp2 datum files

In [ ]:
# Adjust the Copernicus 30m product to be in heights above ellipsoid instead of EGM2008 geoid
# dem_geoid bug fixed in ASP 3.2.1+
!/mnt/1.0_TB_VOLUME/sethv/dshean_tools/StereoPipeline-3.2.1-alpha-2023-05-11-x86_64-Linux/bin/dem_geoid --reverse-adjustment --geoid EGM2008 output_COP30.tif


In [ ]:
refdem_from_opentopography_ellipsoid = refdem_from_opentopography.replace(".tif", "-adj.tif")
!gdalwarp -overwrite -r cubic -t_srs "EPSG:32610" -te $xmin $ymin $xmax $ymax $refdem_from_opentopography_ellipsoid $refdem


In [ ]:
t_srs = "EPSG:32610"  # add vertical CRS if compatible with ASP?

In [ ]:
refdem_rxr = rioxarray.open_rasterio(refdem)
refdem_opentopography_rxr = rioxarray.open_rasterio(refdem_from_opentopography)
diff = refdem_rxr - refdem_opentopography_rxr.rio.reproject_match(refdem_rxr)
print(f"Refdem in EGM2008 heights minimum elevation: {refdem_opentopography_rxr.min().item()}")
print(f"Adjusted refdem minimum elevation: {refdem_rxr.min().item()}")


**Plot the difference after vertical datum adjustment - pixel shape artifacts expected since adjusted refdem is interpolated and raw refdem is not.**

In [ ]:
import matplotlib.pyplot as plt

diff.plot(vmin=-30, vmax=30, cmap="RdBu")
plt.title(f"Mean difference after datum change (EGM2008 -> WGS84): {diff.mean().item():.2f} m")


## Align sparse point cloud (which comes from initial bundle adjustment) to Copernicus DEM
* Note: this should not result in much vertical translation, but some horizontal translation

In [ ]:
sparse_cloud_fn = "dg_csm_model_refined/run-final_residuals_pointmap.csv"

In [ ]:
!pc_align --max-displacement 20 --csv-format "2:lat 1:lon 3:height_above_datum" --save-transformed-source-points $refdem $sparse_cloud_fn -o align_sparse_ba_cloud_to_cop30/align_sparse_ba_cloud_to_cop30 


## Update camera models for mapproject

In [ ]:
left_adjust_fn = f"dg_csm_model_refined/run-{left_image_id}.r100.adjusted_state.json"
right_adjust_fn = f"dg_csm_model_refined/run-{right_image_id}.r100.adjusted_state.json"
!bundle_adjust -t csm --apply-initial-transform-only yes $left_image_fn $right_image_fn \
    $left_adjust_fn $right_adjust_fn \
    --initial-transform align_sparse_ba_cloud_to_cop30/align_sparse_ba_cloud_to_cop30-transform.txt \
    -o csm_cameras_aligned_to_copernicus/csm_cameras_aligned_to_copernicus

## Orthorectification

In [ ]:
# lowres = 1.0 # meters
# left_orthoimage_lowres_fn = f"ortho_left_{lowres}.tif"
# !mapproject -t csm --ot UInt16 --t_srs $t_srs --t_projwin $xmin $ymin $xmax $ymax \
#      --tr $lowres \
#     $refdem \
#     $left_image_fn $left_adjust_fn $left_orthoimage_lowres_fn

In [ ]:
left_bundle_adjusted_and_aligned_csm_fn = f"csm_cameras_aligned_to_copernicus/csm_cameras_aligned_to_copernicus-run-{left_image_id}.r100.adjusted_state.json"
right_bundle_adjusted_and_aligned_csm_fn = f"csm_cameras_aligned_to_copernicus/csm_cameras_aligned_to_copernicus-run-{right_image_id}.r100.adjusted_state.json"


## Mapprojection at full resolution

In [ ]:
left_orthoimage_fullres_fn = f"ortho_left_{res}m.tif"
!mapproject -t csm --ot UInt16 --t_srs $t_srs \
     --t_projwin $xmin $ymin $xmax $ymax \
     --tr $res \
    $refdem \
    $left_image_fn $left_bundle_adjusted_and_aligned_csm_fn $left_orthoimage_fullres_fn

In [ ]:
right_orthoimage_fullres_fn = f"ortho_right_{res}m.tif"
!mapproject -t csm --ot UInt16 --t_srs $t_srs \
     --t_projwin $xmin $ymin $xmax $ymax \
     --tr $res \
    $refdem \
    $right_image_fn $right_bundle_adjusted_and_aligned_csm_fn $right_orthoimage_fullres_fn

## Best parallel_stereo command for run 2: same parameters, correctly reprojected and interpolated refdem
* Explanations of stereo parameters go in the README

In [ ]:
# Running this in Tmux
!parallel_stereo \
    --corr-kernel 7 7 \
    --cost-mode 3  \
    --subpixel-kernel 15 15 \
    --subpixel-mode 9 \
    --stereo-algorithm asp_mgm \
    --alignment-method none \
    --num-matches-from-disparity 10000 \
    --corr-tile-size 1024 \
    --corr-memory-limit-mb 5000 \
    --erode-max-size 0 \
    ortho_left_0.541m.tif \
    ortho_right_0.541m.tif \
    csm_cameras_aligned_to_copernicus/csm_cameras_aligned_to_copernicus-run-1020010042D39D00.r100.adjusted_state.json \
    csm_cameras_aligned_to_copernicus/csm_cameras_aligned_to_copernicus-run-1020010043455300.r100.adjusted_state.json \
    stereo_processing/run_large_2 \
    refdem_copernicus_30m_32610_wgs84.tif

## DEM quality check before coregistration: create DEMs at different grid sizes

In [ ]:
# Can also try 2 or 3 meter grid and resample for smoother results if needed?
# !point2dem --tr 2.0 stereo_processing/run_large_2-PC.tif -o "stereo_processing/run_large_2-2.0m"
!point2dem --tr 1.0 --errorimage stereo_processing/run_large_2-PC.tif -o "stereo_processing/run_large_2-1.0m"

## Align stereo point cloud to bare-ground filtered lidar point cloud

In [ ]:
lidar_reference_bareground_laz = "/mnt/1.0_TB_VOLUME/sethv/resdepth_all/deep-elevation-refinement/ResDepth/torchgeo_experiments/usgs_all616_laz_filtered_dem_mask_nlcd_rock_exclude_glaciers/merged/merged_baker_bareground_all.laz"


In [ ]:
# Run in shell/script, very slow for notebook use
!pc_align --max-displacement 30 --save-transformed-source-points --compute-translation-only \
$lidar_reference_bareground_laz \
stereo_processing/run_large_2-PC.tif \
-o stereo_processing/translation_alignment/run  \
--highest-accuracy


In [ ]:
# TODO split out & fix exact extent
aligned_stereo_pc = "stereo_processing/translation_alignment/run-trans_source.tif"
aligned_stereo_dem_prefix = "stereo_processing/translation_alignment/aligned_stereo_1.0m"
aligned_stereo_dem_fn = aligned_stereo_dem_prefix + "-DEM.tif"
!point2dem --tr 1 --t_srs $t_srs --errorimage --nodata-value -9999 \
$aligned_stereo_pc -o stereo_processing/translation_alignment/aligned_stereo_1.0m

## Align self-consistent CSM camera models to lidar


In [ ]:
!bundle_adjust -t csm --apply-initial-transform-only yes \
$left_image_fn \
$right_image_fn \
"$left_bundle_adjusted_and_aligned_csm_fn" \
"$right_bundle_adjusted_and_aligned_csm_fn" \
--initial-transform stereo_processing/translation_alignment/run-transform.txt \
-o stereo_processing/translation_alignment/lidar_aligned_csm_cameras

In [ ]:
output_res = 1.0


In [ ]:
left_final_aligned_csm_fn = f"stereo_processing/translation_alignment/lidar_aligned_csm_cameras-csm_cameras_aligned_to_copernicus-run-{left_image_id}.r100.adjusted_state.json"
right_final_aligned_csm_fn = f"stereo_processing/translation_alignment/lidar_aligned_csm_cameras-csm_cameras_aligned_to_copernicus-run-{right_image_id}.r100.adjusted_state.json"


In [ ]:
right_orthoimage_fullres_fn = f"final_ortho_right_{output_res:.1f}m.tif"

!mapproject -t csm --ot UInt16 --t_srs $t_srs \
 --t_projwin $xmin $ymin $xmax $ymax \
 --tr $output_res \
$aligned_stereo_dem_fn $right_image_fn $right_final_aligned_csm_fn $right_orthoimage_fullres_fn

In [ ]:
left_orthoimage_fullres_fn = f"final_ortho_left_{output_res:.1f}m.tif"

!mapproject -t csm --ot UInt16 --t_srs $t_srs \
 --t_projwin $xmin $ymin $xmax $ymax \
 --tr $output_res \
$aligned_stereo_dem_fn $left_image_fn $left_final_aligned_csm_fn $left_orthoimage_fullres_fn

## QGIS observation: orthoimages are now self-consistent and match the aligned stereo DEM!

## Last step: rasterize lidar on same grid
* Use point2dem to get lidar DEM with same gridding and pixel centers as the stereo DEM - pdal will not match exactly

## Problem to resolve: the "lidar-aligned" stereo DEM and orthoimages are self-consistent but do not line up with lidar DEM?!

In [ ]:
# TODO to produce lidar DEM have to redo filtering & merging of entire Baker lidar point cloud
# into a massive single point cloud rather than USGS tiles.
# For now, bare ground points used for alignment result in subset DEM that aligns w/ stereo DEM
# lidar_pc="/mnt/1.0_TB_VOLUME/sethv/resdepth_all/deep-elevation-refinement/ResDepth/torchgeo_experiments/"

!point2dem --tr $output_res --t_srs $t_srs --nodata-value -9999 \
--t_projwin $xmin $ymin $xmax $ymax \
$lidar_reference_bareground_laz -o "lidar_bareground_only_DEM_test${output_res}"

In [ ]:
lidar_reference_bareground_dem="lidar_bareground_only_DEM_test.0-DEM.tif"

**Check that using DEM raster makes no significant difference vs lidar point cloud**

In [ ]:
# pc_align transforms when aligning to (patchy) bare ground point cloud
# (-2.1737324,10.257786,4.0624801) point cloud
# (-2.2497165,10.18713,4.0598247) DEM within target bbox
!pc_align --max-displacement 30 --save-transformed-source-points --compute-translation-only \
$lidar_reference_bareground_dem \
stereo_processing/run_large_2-PC.tif \
-o try_pc_align_to_lidar_dem_not_pc/run  \
--highest-accuracy

In [ ]:
# show with USGS_LPC_WA_MtBaker_2015_10UEU8599_LAS_2017
# should be aligned with the bare ground lidar?
example_tile_in_fn = "/mnt/Backups/sethv/USGS_LPC_WA_MtBaker_2015/USGS_LPC_WA_MtBaker_2015_10UEU8599_LAS_2017.laz"
example_tile_fn = "example_v1.3.laz"
!pdal translate -i $example_tile_in_fn -o $example_tile_fn --writers.las.format="1.3"

!point2dem --tr $output_res --t_srs $t_srs --nodata-value -9999 \
    $example_tile_fn \
    -o "example_lidar_tile"